In [23]:
import sys
sys.path.append("../Pupillengröße/Skripte/")
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging as log
from numpy.fft import fft

from preprocessfunction2 import PLR2d
from preprocessfunction3 import PLR3d
import checkdata
import importlib
from IPython.display import display
def display_all(df):
    from IPython.display import HTML, display
    IPython.display(IPython.HTML(df.to_html()))


In [47]:
import preprocessing
import plotting
import math
import os

importlib.reload(preprocessing)
importlib.reload(plotting)
importlib.reload(checkdata)

def plot(df,text,xcol,ycol):
    plotting.plot_preprocessed(df,text,xcol,ycol)
    
def noplot(df,text,xcol,ycol):
    pass
    
if os.getlogin()=='marco':
    subject_id="PJ15_1_Ruhe"
    data_dir="/home/marco/Nextcloud/KatharinaBeispieldaten/"
else:
    subject_id="PJ02_1_Ruhe"
    data_dir="/Users/Katharina/Desktop/Beispieldaten"
    
config=preprocessing.create_process_config(0,"diameter",subject_id,"Ruhe","30",data_dir)
config.validate_only=True

eye0=preprocessing.process(config,display)
display(eye0)
if False:
    for what in eye0:
        ts=what['pupil_timestamp'].iloc[0]
        plot(what,f"frame @{ts}","pupil_timestamp_based","diameter_original")
        plot(what,f"recon @{ts}","pupil_timestamp_based","diameter_rec")
        plot(what,f"interp @{ts}","pupil_timestamp_based","diameter_rec_interp_100")

    if False:
        for what in eye0:
            plotting.plot_preprocessed(what,f"frame @what.pupil_timestamp[0]","pupil_timestamp_based","diameter_gated")

    display(f"Anzahl der Messungen: {len(eye0)}")
    if True:
        sub=None
        i=0
        for what in eye0:
            preprocessing.create_baseline_column(what, 'diameter_original','diameter_original_baseline')
            sub = what.plot(ax=sub,x='pupil_timestamp_based', y='diameter_baseline',ylabel=f'baseline adjusted pupil size', kind='line')        
            sub.set_ylabel(f'measurement {i}, diameter')
            i=i+1



'Starting process2'

'Loop through each annotation timestamp and slice the data'

'measurement @28644.466044 has 7.927876359473383% noise data. Rejecting'

'measurement @29484.472736 has 6.519308506063473% noise data. Rejecting'

[('PJ15_1_Ruhe', 28344.482733, 2.6982707215265354, True),
 ('PJ15_1_Ruhe', 28404.46642, 1.8145624377918996, True),
 ('PJ15_1_Ruhe', 28464.482395, 0.921937580291695, True),
 ('PJ15_1_Ruhe', 28524.466376, 0.783229670582815, True),
 ('PJ15_1_Ruhe', 28584.482233, 0.781010719754977, True),
 ('PJ15_1_Ruhe', 28644.466044, 7.927876359473383, False),
 ('PJ15_1_Ruhe', 28704.486049, 0.9380004575611988, True),
 ('PJ15_1_Ruhe', 28764.465902, 1.0571472345641182, True),
 ('PJ15_1_Ruhe', 28824.485859, 1.1709421462136969, True),
 ('PJ15_1_Ruhe', 28884.469922, 1.6978599068151308, True),
 ('PJ15_1_Ruhe', 28944.4856, 2.7314814814814814, True),
 ('PJ15_1_Ruhe', 29004.469644, 2.3390013079941525, True),
 ('PJ15_1_Ruhe', 29064.453515, 2.1254740834386854, True),
 ('PJ15_1_Ruhe', 29124.469325000002, 3.8810835985407124, True),
 ('PJ15_1_Ruhe', 29184.453265, 3.098521349061306, True),
 ('PJ15_1_Ruhe', 29244.469168, 2.5196850393700787, True),
 ('PJ15_1_Ruhe', 29304.453082, 1.6021247429746401, True),
 ('PJ15_1_Ruhe'

In [ ]:
### Use datamatrix from pydatamatrix.eu to fill in blinking gaps